In [1]:
import numpy as np
import pandas as pd
import healpy as hp

In [2]:
filepath = '/Users/jimenagonzalez/research/DSPL/Hybrid_SL_Modeling/data/candidates_tocompare/Rojas/'

# Reading table1 - A SL candidates
colspecs = [(0,18), (19,39), (40,61), (62,66), (67,71), (72,101)]
table1 = pd.read_fwf( filepath + "table1.dat", colspecs=colspecs, 
                     names=['Cand', 'RA', 'DEC', 'S_ML', 'S_VI', 'Ref'])

# Reading table2 - B&C SL candidates
colspecs = [(0,18), (19,39), (40,61), (62,66), (67,71), (72,101)]
table2 = pd.read_fwf( filepath + "table2.dat", colspecs=colspecs, 
                     names=['Cand', 'RA', 'DEC', 'S_ML', 'S_VI', 'Ref'])

table2.head()

,Cand,RA,DEC,S_ML,S_VI,Ref
0,DES J034130-513044,55.378331,-51.512411,1.0,1.0,"M7, M10"
1,DES J034744-245431,56.935562,-24.908741,1.0,1.0,"M9, M10"
2,DES J044408-655430,71.034707,-65.908598,1.0,1.0,This work
3,DES J010548-372542,16.450174,-37.428457,1.0,1.0,M10
4,DES J015138-242628,27.909990,-24.441314,1.0,1.0,M18


In [3]:
# Reading table4 - Modeling results for the candidates
colspecs = [(0,18), (18, 19), (20,24), (25,30), (30, 31), (31,36), (36,37), (37,42), (43,48), (48, 49), (49,54), (54,55), 
(55,60), (61,64), (64, 65), (65,67), (67,68), (68,70), (71,76) , (76,77),  (77,82), (82,83) , (83,88 ), (89,92), 
(92,93), (93,95),  (95,96) , (96,98)]
table4 = pd.read_fwf(filepath + "table4.dat", colspecs=colspecs, 
                     names=['Cand', 'n_Cand', 'rchi2', 'RE', '1', 'E_RE', '2', 'e_RE', 'qm', '3',
                            'E_qm', '4', 'e_qm', 'PAm', '5', 'E_PAm', '6', 'e_PAm', 'gammaext', '7', 
                            'E_gammaext', '8', 'e_gammaext', 'phiext', '9', 'E_phiext', '10', 'e_phiext'])
table4 = table4.drop(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], axis=1)

table4.head()

,Cand,n_Cand,rchi2,RE,E_RE,e_RE,qm,E_qm,e_qm,PAm,E_PAm,e_PAm,gammaext,E_gammaext,e_gammaext,phiext,E_phiext,e_phiext
0,DES J003727-413149,NaN,1.08,2.315,0.004,0.004,0.685,0.011,0.013,23,2,3,0.159,0.004,0.004,-9,1,1
1,DES J060653-585843,NaN,1.92,2.401,0.006,0.005,0.692,0.011,0.012,79,3,2,0.222,0.003,0.002,-84,1,1
2,DES J060653-585843,*,1.14,2.063,0.029,0.017,0.904,0.024,0.034,14,18,14,0.074,0.011,0.004,14,8,3
3,DES J042218-213245,NaN,1.43,2.116,0.013,0.012,0.732,0.019,0.019,29,5,5,0.103,0.008,0.008,-85,5,5
4,DES J045901-204506,NaN,1.10,1.684,0.004,0.004,0.801,0.014,0.015,-14,5,5,0.213,0.004,0.004,-21,1,1


In [4]:
print(len(table1), len(table2), len(table4))

#They only modeled A category lenses
new_table = table4.merge(table1[['Cand', 'RA', 'DEC', 'S_VI']], on='Cand')
new_table['Rank_norm'] = 3 * (new_table['S_VI'] - new_table['S_VI'].min()) / (new_table['S_VI'].max() - new_table['S_VI'].min())
new_table = new_table.rename({'Cand': 'Name', 'Rank_norm': 'Exp_score', 'E_RE': 'U_RE', 'e_RE': 'L_RE', 'E_qm': 'U_qm', 
                              'e_qm': 'L_qm', 'E_PAm': 'U_PAm', 'e_PAm': 'L_PAm', 'E_gammaext': 'U_gammaext', 
                              'e_gammaext': 'L_gammaext', 'E_phiext': 'U_phiext', 'e_phiext': 'L_phiext'}, axis=1)
new_table = new_table[['Name', 'RA', 'DEC', 'Exp_score', 'n_Cand', 'rchi2', 'RE', 'U_RE', 'L_RE', 'qm', 'U_qm', 
            'L_qm', 'PAm', 'U_PAm', 'L_PAm', 'gammaext', 'U_gammaext', 'L_gammaext', 'phiext', 'U_phiext', 
            'L_phiext']]

#Adding healpix
ra_rad, dec_rad = np.deg2rad(new_table['RA'].values), np.deg2rad(new_table['DEC'].values)
theta, phi = 0.5 * np.pi - dec_rad, ra_rad 
new_table['HPIX_4096'] = hp.ang2pix(4096, theta, phi, nest=True)

#Removing potential issues
new_table = new_table.fillna(' ')
new_table = new_table.astype({'Name': str, 'RA': float, 'DEC': float, 'Exp_score': float, 'n_Cand': str, 
        'rchi2': float, 'RE': float, 'U_RE': float, 'L_RE': float, 'qm': float, 'U_qm': float, 'L_qm': float, 
        'PAm': float, 'U_PAm': float, 'L_PAm': float, 'gammaext': float, 'U_gammaext': float, 'L_gammaext': float,
        'phiext': float, 'U_phiext': float, 'L_phiext': float, 'HPIX_4096': int})

new_table.to_csv('Initial_Rojas.csv', index=False)
new_table.head()

90 315 61


,Name,RA,DEC,Exp_score,n_Cand,rchi2,RE,U_RE,L_RE,qm,...,PAm,U_PAm,L_PAm,gammaext,U_gammaext,L_gammaext,phiext,U_phiext,L_phiext,HPIX_4096
0,DES J003727-413149,9.362803,-41.530542,3.0,,1.08,2.315,0.004,0.004,0.685,...,23.0,2.0,3.0,0.159,0.004,0.004,-9.0,1.0,1.0,145321235
1,DES J060653-585843,91.721422,-58.978786,3.0,,1.92,2.401,0.006,0.005,0.692,...,79.0,3.0,2.0,0.222,0.003,0.002,-84.0,1.0,1.0,159917359
2,DES J060653-585843,91.721422,-58.978786,3.0,*,1.14,2.063,0.029,0.017,0.904,...,14.0,18.0,14.0,0.074,0.011,0.004,14.0,8.0,3.0,159917359
3,DES J042218-213245,65.575901,-21.546084,3.0,,1.43,2.116,0.013,0.012,0.732,...,29.0,5.0,5.0,0.103,0.008,0.008,-85.0,5.0,5.0,142588074
4,DES J045901-204506,74.756099,-20.751891,3.0,,1.10,1.684,0.004,0.004,0.801,...,-14.0,5.0,5.0,0.213,0.004,0.004,-21.0,1.0,1.0,86583907
